In [1]:
import pymysql
import streamlit as st

@st.cache_resource 
def get_tidb_connection(): 
    return pymysql.connect( 
        host=st.secrets["tidb"]["host"], 
        port=st.secrets["tidb"]["port"], 
        user=st.secrets["tidb"]["user"], 
        password=st.secrets["tidb"]["password"], 
        ssl={"ssl": True} ) 

mydb = get_tidb_connection() 
mycursor = mydb.cursor()


2025-12-24 11:28:10.562 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-24 11:28:11.677 
  command:

    streamlit run f:\Guvi\Projects\IMDB\Imdb_venv\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-12-24 11:28:11.679 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-24 11:28:11.680 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-24 11:28:11.682 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-24 11:28:12.215 Thread 'Thread-3': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-24 11:28:12.281 Thread 'Thread-3': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-24 11:28:12.289 Thread 'Thread-3': m

In [12]:
mycursor.execute("CREATE DATABASE IF NOT EXISTS IMDB")

0

In [44]:
import pandas as pd

df = pd.read_csv("../data/imdb_movie_list_Cleaned.csv")

In [45]:
df = df[df['Duration'] <= 250]

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11581 entries, 0 to 11588
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Title     11581 non-null  object 
 1   Ratings   11581 non-null  float64
 2   Votings   11581 non-null  int64  
 3   Duration  11581 non-null  int64  
 4   Genre     11581 non-null  object 
dtypes: float64(1), int64(2), object(2)
memory usage: 542.9+ KB


In [ ]:
mycursor.execute("USE IMDB")

mycursor.execute("""CREATE TABLE IF NOT EXISTS imdb_movies_list(
                 ID int auto_increment primary key,
                 Title varchar(255),
                 Ratings float,
                 Votings int,
                 Duration int,
                 Genre varchar(100))
""")

for _, row in df.iterrows():
    sql = "INSERT INTO imdb_movies_list (Title, Ratings, Votings, Duration, Genre) VALUES (%s, %s, %s, %s, %s)"
    val = (row['Title'], row['Ratings'], row['Votings'], row['Duration'], row['Genre'])
    mycursor.execute(sql, val)
    mydb.commit()

In [ ]:
mycursor.execute("""CREATE TABLE IMDB.images (
    id INT AUTO_INCREMENT PRIMARY KEY,
    image_name VARCHAR(255),
    image_data LONGBLOB)""")

0

In [26]:
# Read image as binary
image_path = "../assets/Imdb_Image.png"
with open(image_path, "rb") as file:
    image_binary = file.read()

# Insert image into TiDB
sql = """INSERT INTO IMDB.images (image_name, image_data)VALUES (%s, %s)"""

mycursor.execute(sql, ("Imdb_Image.png", image_binary))
mydb.commit()

In [46]:
df.to_csv('../data/imdb_movie_list_Cleaned.csv', index=False)